# Warehouse Problem

We will implement the model studied during the course.

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c} & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} = 1 & \forall c \\
&\sum_{c} y_{w,c} \leq C_w x_w & \forall w \\
&x_w, y_{w,c} \in \mathbb{B} & \forall w,c
\end{align*}


In [5]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]


nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)

### Exercices

#### 1. Modify the model to add: a capacity on the number of clients for the warehouses and a penalty if a client is not served


#### 2. Implement the model

## Adding the capacity constraint

\begin{align*}
\min &\qquad \sum_w f_w x_w + \sum_{w, c} t_{w,c} y_{w,c} + M \sum_c z_c & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w, c \\
&\sum_w y_{w,c} + z_c = 1 & \forall c \\
&\sum_{c} y_{w,c} \leq \text{capacity}_w \cdot x_w & \forall w \\
&x_w, y_{w,c}, z_c \in \{0, 1\} & \forall w, c
\end{align*}



In [6]:
from docplex.mp.model import Model

# Create a model
model = Model(name='warehouse')

# Decision variables
x = model.binary_var_list(nbWarehouses, name='open')  # x[w] = 1 if warehouse w is open
y = model.binary_var_matrix(nbStores, nbWarehouses, name='supplier')  # y[s, w] = 1 if store s is served by warehouse w
z = model.binary_var_list(nbStores, name='not_served')  # z[s] = 1 if store s is not served

# Objective function: minimize total cost (opening + transportation + penalty)
model.minimize(
    model.sum(fixed_cost * x[w] for w in range(nbWarehouses)) +  # fixed cost for opening warehouses
    model.sum(supplyCost[s][w] * y[s, w] for s in range(nbStores) for w in range(nbWarehouses)) +  # transportation cost
    M * model.sum(z[s] for s in range(nbStores))  # penalty for unserved customers
)

# Constraints: if a warehouse is not open, it cannot supply any store
for w in range(nbWarehouses):
    for s in range(nbStores):
        model.add_constraint(y[s, w] <= x[w], f'supply_{s}_{w}')  # If warehouse w is closed, it cannot serve any store

# Constraints: each store must be served by one warehouse or marked as unserved
for s in range(nbStores):
    model.add_constraint(model.sum(y[s, w] for w in range(nbWarehouses)) + z[s] == 1, f'store_{s}')  # Each store must be served or unserved

# Constraints: each warehouse cannot serve more stores than its capacity
for w in range(nbWarehouses):
    model.add_constraint(model.sum(y[s, w] for s in range(nbStores)) <= capacity[w] * x[w], f'capacity_{w}')  # Respect warehouse capacity



Objective value:  383.0

Warehouses to be opened:
 - Warehouse 0 is open with capacity 1
 - Warehouse 1 is open with capacity 4
 - Warehouse 2 is open with capacity 2
 - Warehouse 4 is open with capacity 3

Store assignments:
Store 0 is served by Warehouse(s): [4]
Store 1 is served by Warehouse(s): [1]
Store 2 is served by Warehouse(s): [4]
Store 3 is served by Warehouse(s): [0]
Store 4 is served by Warehouse(s): [4]
Store 5 is served by Warehouse(s): [1]
Store 6 is served by Warehouse(s): [1]
Store 7 is served by Warehouse(s): [2]
Store 8 is served by Warehouse(s): [1]
Store 9 is served by Warehouse(s): [2]


In [7]:
# Solve the model
solution = model.solve()

# Print the solution
if solution:
    print("Objective value: ", model.objective_value)
    
    print("\nWarehouses to be opened:")
    for w in range(nbWarehouses):
        if x[w].solution_value > 0.5:  # Warehouse is open
            print(f" - Warehouse {w} is open with capacity {capacity[w]}")
    
    print("\nStore assignments:")
    for s in range(nbStores):
        assigned = [w for w in range(nbWarehouses) if y[s, w].solution_value > 0.5]
        if assigned:
            print(f"Store {s} is served by Warehouse(s): {assigned}")
        else:
            print(f"Store {s} is not served (penalty applied)")
else:
    print("No feasible solution found.")

Objective value:  383.0

Warehouses to be opened:
 - Warehouse 0 is open with capacity 1
 - Warehouse 1 is open with capacity 4
 - Warehouse 2 is open with capacity 2
 - Warehouse 4 is open with capacity 3

Store assignments:
Store 0 is served by Warehouse(s): [4]
Store 1 is served by Warehouse(s): [1]
Store 2 is served by Warehouse(s): [4]
Store 3 is served by Warehouse(s): [0]
Store 4 is served by Warehouse(s): [4]
Store 5 is served by Warehouse(s): [1]
Store 6 is served by Warehouse(s): [1]
Store 7 is served by Warehouse(s): [2]
Store 8 is served by Warehouse(s): [1]
Store 9 is served by Warehouse(s): [2]


In [8]:
# Calculate total capacity of open warehouses
total_capacity_open = sum(capacity[w] for w in range(nbWarehouses) if x[w].solution_value > 0.5)

print(f"Total capacity of open warehouses: {total_capacity_open}")

Total capacity of open warehouses: 10
